<a href="https://colab.research.google.com/github/VishvalFadatare/Python/blob/main/Copy_of_EXL_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EXL Round 2 – Student Report Template

## 1. Introduction
Provide a short description of the project, objective, and your overall approach.

## 2. Dataset Understanding
Perform Operations &
Describe each dataset with row counts and key issues found:
- **users.csv**:
- **products.csv**:
- **orders.csv**:
- **order_items.csv**:

## 3. Data Cleaning & Preprocessing
Explain the cleaning steps applied:
- Null handling
- Deduplication
- Fixing inconsistent formatting
- Standardizing date formats
- Data type conversions
- City/category/SKU standardization
- Validating foreign keys

## 4. Data Quality Checks
List the DQ checks performed and results **before and after** cleaning:
- Duplicate check
- Null value check
- Invalid foreign key detection
- Numeric field validation
- Timestamp validation

## 5. Data Modeling (Star Schema)
Insert your ERD diagram image and description.

**Example tables:**
- fact_orders
- dim_users
- dim_products
- dim_date

## 6. ETL Pipeline Explanation
Explain the ETL pipeline steps:
- Input ingestion
- Cleaning transformations
- Joins & schema creation
- Fact and dimension population

## 7. SQL Analytical Queries & Results
Perform Analytical Queries.

## 8. Final Summary
Final comments, learnings, challenges faced, and improvement suggestions.

**Install & Create Spark Session**

In [ ]:
!pip install pyspark


In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Ecommerce Cleaning & Star Schema") \
    .getOrCreate()


**Upload CSV Files**

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving order_items.csv to order_items.csv
Saving orders.csv to orders.csv
Saving products.csv to products.csv
Saving users.csv to users.csv


**Load CSV Files into Spark**

In [ ]:
users_df = spark.read.csv("users.csv", header=True, inferSchema=True)
products_df = spark.read.csv("products.csv", header=True, inferSchema=True)
orders_df = spark.read.csv("orders.csv", header=True, inferSchema=True)
order_items_df = spark.read.csv("order_items.csv", header=True, inferSchema=True)


Check schemas:

In [ ]:
users_df.printSchema()
products_df.printSchema()
orders_df.printSchema()
order_items_df.printSchema()


root
 |-- user_id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- city: string (nullable = true)

root
 |-- product_id: integer (nullable = true)
 |-- sku: string (nullable = true)
 |-- product_name: string (nullable = true)
 |-- category: string (nullable = true)

root
 |-- order_id: integer (nullable = true)
 |-- user_id: double (nullable = true)
 |-- order_timestamp: string (nullable = true)
 |-- amount: string (nullable = true)

root
 |-- order_id: integer (nullable = true)
 |-- order_item_id: integer (nullable = true)
 |-- sku: string (nullable = true)
 |-- price: string (nullable = true)
 |-- quantity: string (nullable = true)



**Data Cleaning**

Remove NULL Values

In [ ]:
users_df = users_df.dropna()
products_df = products_df.dropna()
orders_df = orders_df.dropna()
order_items_df = order_items_df.dropna()


Remove Duplicate Records

In [ ]:
users_df = users_df.dropDuplicates()
products_df = products_df.dropDuplicates()
orders_df = orders_df.dropDuplicates()
order_items_df = order_items_df.dropDuplicates()


Fix Wrong Data Types

In [ ]:
from pyspark.sql.functions import col

order_items_df = order_items_df \
    .withColumn("price", col("price").cast("double")) \
    .withColumn("quantity", col("quantity").cast("int"))

orders_df = orders_df \
    .withColumn("amount", col("amount").cast("double"))


Remove Zero / Invalid Values

In [ ]:
users_df = users_df.filter(col("user_id").isNotNull() & (col("user_id") != 0))
products_df = products_df.filter(col("sku").isNotNull())


In [ ]:
users_df.show()


+-------+------------------+-------------+
|user_id|              name|         city|
+-------+------------------+-------------+
|   2841|        Sneha  Roy|       Jaipur|
|   4781|  Mahesh Agarwal  |     Amritsar|
|   1403|    Olivia chauhan|   Aurangabad|
|   1327|       Maria Gowda|     Srinagar|
|   1722|         Emma Jain|       Meerut|
|   3842|siddharth Thompson|         Agra|
|   2697|       EMMA Sharma|        Kochi|
|   1609|      HARSH Mishra|        delhi|
|   1626|  Emma Fernandes  |        Surat|
|   4523|       Liam Khanna|   Aurangabad|
|   3263|       Emma  Gupta|        delhi|
|   1802|    Peter Ansari  |   Dhanbad   |
|   4420|    Gaurav Joshi  |    Ghaziabad|
|   3577|      Sara Reddy  |       Jaipur|
|   4998|     Rhea  Chauhan|   Vijayawada|
|   3250|      Farhan Dutta|        deLHi|
|   4284|     VIVEK  Kapoor|   CoimbAtore|
|   3844|   Suresh Tripathi|    Ghaziabad|
|    715|     Harsh agarwal|     Srinagar|
|   3167|        Aman  Khan|        delhi|
+-------+--

**Create Star Schema**

Dimension Tables

In [ ]:
dim_users = users_df.select("user_id", "name", "city")
dim_products = products_df.select("sku", "product_name", "category")


Fact Table

In [ ]:
fact_orders = order_items_df \
    .join(orders_df, "order_id") \
    .select("order_id", "sku", "price", "quantity", "user_id")


Register SQL views:

In [ ]:
dim_users.createOrReplaceTempView("dim_users")
dim_products.createOrReplaceTempView("dim_products")
fact_orders.createOrReplaceTempView("fact_orders")


View Star Schema Using SQL

In [ ]:
spark.sql("""
SELECT f.order_id, u.name, p.product_name, f.price, f.quantity
FROM fact_orders f
JOIN dim_users u ON f.user_id = u.user_id
JOIN dim_products p ON f.sku = p.sku
""").show()


+--------+-------------------+--------------+------+--------+
|order_id|               name|  product_name| price|quantity|
+--------+-------------------+--------------+------+--------+
|    3983|   Gaurav  Williams|         CHAIR|  NULL|       2|
|   15945|      GAURAV Taylor|  Smartwatch  |1096.0|    NULL|
|   16823|          Amit khan|      Tablet  |  NULL|       2|
|   16483|        neha Taylor| graphics card|1990.0|       1|
|    6575|     kavita D'Souza|     Monitor  |999.99|    NULL|
|   18078|    Priya fernandes|      sneakers|2589.0|       2|
|   10713|      OLIVIA Sheikh|         Chair|4995.0|       1|
|    5106|     Anita Sharma  |        Jacket|999.99|       1|
|    7348|   siddharth Sheikh|    LED Bulb  |2100.0|       2|
|   17549|        James gupta|  FICTION BOOK| 100.0|    NULL|
|    1414|         VIVEK Khan|       monitor|999.99|       2|
|    6891|       noah Agarwal|         Bag  |3190.0|    NULL|
|   15518|     Komal anderson|      NOTEBOOK|4032.0|       2|
|   1093

**Spark SQL Queries**

Revenue per product:

In [ ]:
spark.sql("""
SELECT sku, SUM(price * quantity) AS revenue
FROM fact_orders
GROUP BY sku
ORDER BY revenue DESC
""").show()


+-----------+------------------+
|        sku|           revenue|
+-----------+------------------+
|   SKU677  |           28717.0|
|     SKU783|           26356.0|
|     SKU660|          25769.98|
|   SKU135  |           24624.0|
|  SKU1067  |24167.989999999998|
|    SKU_886|          23871.98|
|   sku-554 |           23568.0|
|      SKU30|           22196.0|
|     SKU559|          21839.98|
|    SKU_857|           21793.0|
|   SKU116  |           21352.0|
|     SKU344|           21218.0|
|    SKU_452|           20790.0|
|    sku-70 |           20777.0|
|     SKU473|          20707.96|
|   SKU362  |           20551.0|
|   SKU367  |           20434.0|
|     SKU404|          20274.98|
|     SKU503|           20260.0|
|  sku-1192 |20181.989999999998|
+-----------+------------------+
only showing top 20 rows



Total quantity:

In [ ]:
spark.sql("""
SELECT sku, SUM(quantity) AS total_qty
FROM fact_orders
GROUP BY sku
""").show()


+-----------+---------+
|        sku|total_qty|
+-----------+---------+
|    sku-32 |        3|
|     SKU161|        5|
|   sku-240 |       11|
|   sku-955 |        2|
|   SKU778  |        2|
|   SKU649  |        3|
|     SKU781|        4|
|     sku-9 |        2|
|    SKU1146|        4|
|   SKU719  |        1|
|   sku-901 |        5|
|   sku-506 |        2|
|    SKU_549|     NULL|
|   sku-458 |        3|
|    SKU_657|        4|
|  SKU1156  |        1|
|   sku-851 |        2|
|   sku-464 |        1|
|      SKU89|     NULL|
|   SKU266  |     NULL|
+-----------+---------+
only showing top 20 rows



Orders per User

In [ ]:
spark.sql("""
SELECT user_id,
       COUNT(order_id) AS total_orders
FROM fact_orders
GROUP BY user_id
ORDER BY total_orders DESC
""").show()


+-------+------------+
|user_id|total_orders|
+-------+------------+
| 1641.0|          13|
| 4820.0|          12|
| 5051.0|          11|
| 3104.0|          11|
| 5898.0|          11|
| 2326.0|          11|
| 1056.0|          11|
| 1728.0|          10|
| 1997.0|          10|
| 1798.0|          10|
| 5673.0|          10|
| 4056.0|           9|
| 5966.0|           9|
| 3378.0|           9|
|  844.0|           9|
| 1674.0|           9|
| 2437.0|           9|
|  382.0|           9|
| 1115.0|           9|
| 5749.0|           8|
+-------+------------+
only showing top 20 rows



<!-- Add Your Response -->

Top 5 Products by Revenue

In [ ]:
spark.sql("""
SELECT sku,
       SUM(price * quantity) AS revenue
FROM fact_orders
GROUP BY sku
ORDER BY revenue DESC
LIMIT 5
""").show()


+-----------+------------------+
|        sku|           revenue|
+-----------+------------------+
|   SKU677  |           28717.0|
|     SKU783|           26356.0|
|     SKU660|          25769.98|
|   SKU135  |           24624.0|
|  SKU1067  |24167.989999999998|
+-----------+------------------+



Most Sold Product

In [ ]:
spark.sql("""
SELECT sku, SUM(quantity) AS total_quantity
FROM fact_orders
GROUP BY sku
ORDER BY total_quantity DESC
""").show()


+----------+--------------+
|       sku|total_quantity|
+----------+--------------+
|  SKU491  |            13|
|  sku-268 |            13|
|    SKU559|            12|
|  SKU_1126|            12|
|   SKU_607|            12|
|  sku-240 |            11|
|   SKU_248|            11|
|    SKU819|            11|
|    SKU494|            11|
|    SKU322|            10|
|  SKU593  |            10|
|  SKU288  |            10|
|   SKU_786|            10|
|   SKU_409|            10|
|  SKU937  |            10|
|  SKU467  |            10|
|   SKU97  |            10|
|     SKU65|            10|
|    SKU404|            10|
|   SKU_673|            10|
+----------+--------------+
only showing top 20 rows



Products Never Sold

In [ ]:
spark.sql("""
SELECT p.sku, p.product_name
FROM dim_products p
LEFT JOIN fact_orders f
ON p.sku = f.sku
WHERE f.sku IS NULL
""").show()


+--------+-----------------+
|     sku|     product_name|
+--------+-----------------+
|  SKU123|         keyboard|
| SKU556 |           Tablet|
| SKU326 |        Hard Disk|
| SKU315 |         trousers|
| SKU335 |bluetooth speaker|
| SKU460 |       SMARTWATCH|
| sku-747|        USB Cable|
|  sku239|          Charger|
| sku-751|           LAPTOP|
| SKU441 |              RAM|
|  sku923|        PROCESSOR|
| sku-854|          CHARGER|
| SKU924 |             sofa|
|  sku570|        Perfume  |
|   sku15|         Cooker  |
| SKU_968|           tablet|
|  sku728|            Jeans|
| SKU_435|     water bottle|
|  sku630|           Camera|
| SKU607 |     Television  |
+--------+-----------------+
only showing top 20 rows



High Value Orders

In [ ]:
spark.sql("""
SELECT order_id, SUM(price * quantity) AS order_revenue
FROM fact_orders
GROUP BY order_id
HAVING order_revenue >
(
  SELECT AVG(price * quantity)
  FROM fact_orders
)
""").show()


+--------+-----------------+
|order_id|    order_revenue|
+--------+-----------------+
|    8638|          11096.0|
|     471|           7956.0|
|    9900|6258.959999999999|
|    4900|           5712.0|
|    9376|           5916.0|
|   16339|          3999.96|
|   11458|           2940.0|
|   19553|           7214.0|
|    2659|           7876.0|
|    1088|           6230.0|
|     737|          5467.98|
|    1127|          5739.98|
|    1522|          10082.0|
|    2563|          3339.98|
|   19614|           2609.0|
|   16916|          8719.98|
|   11316|           9438.0|
|    1460|          3137.99|
|     858|           3024.0|
|   18201|           4886.0|
+--------+-----------------+
only showing top 20 rows



**Data Quality Checks**

Null Value Check

In [ ]:
from pyspark.sql.functions import col, sum

def null_check(df, name):
    print(f"Null check for {name}")
    df.select([
        sum(col(c).isNull().cast("int")).alias(c)
        for c in df.columns
    ]).show()

null_check(users_df, "Users")
null_check(products_df, "Products")
null_check(orders_df, "Orders")
null_check(order_items_df, "Order Items")


Null check for Users
+-------+----+----+
|user_id|name|city|
+-------+----+----+
|      0|   0|   0|
+-------+----+----+

Null check for Products
+----------+---+------------+--------+
|product_id|sku|product_name|category|
+----------+---+------------+--------+
|         0|  0|           0|       0|
+----------+---+------------+--------+

Null check for Orders
+--------+-------+---------------+------+
|order_id|user_id|order_timestamp|amount|
+--------+-------+---------------+------+
|       0|      0|              0|  1866|
+--------+-------+---------------+------+

Null check for Order Items
+--------+-------------+---+-----+--------+
|order_id|order_item_id|sku|price|quantity|
+--------+-------------+---+-----+--------+
|       0|            0|  0| 4200|    5418|
+--------+-------------+---+-----+--------+



Duplicate Records Check

In [ ]:
print("Duplicate users:", users_df.count() - users_df.dropDuplicates().count())
print("Duplicate products:", products_df.count() - products_df.dropDuplicates().count())
print("Duplicate orders:", orders_df.count() - orders_df.dropDuplicates().count())
print("Duplicate order items:", order_items_df.count() - order_items_df.dropDuplicates().count())


Duplicate users: 0
Duplicate products: 0
Duplicate orders: 0
Duplicate order items: 0


Invalid Numeric Data Check

In [ ]:
orders_df.select("amount").where(col("amount").cast("double").isNull()).show()

order_items_df.select("price").where(col("price").cast("double").isNull()).show()
order_items_df.select("quantity").where(col("quantity").cast("int").isNull()).show()


+------+
|amount|
+------+
|  NULL|
|  NULL|
|  NULL|
|  NULL|
|  NULL|
|  NULL|
|  NULL|
|  NULL|
|  NULL|
|  NULL|
|  NULL|
|  NULL|
|  NULL|
|  NULL|
|  NULL|
|  NULL|
|  NULL|
|  NULL|
|  NULL|
|  NULL|
+------+
only showing top 20 rows

+-----+
|price|
+-----+
| NULL|
| NULL|
| NULL|
| NULL|
| NULL|
| NULL|
| NULL|
| NULL|
| NULL|
| NULL|
| NULL|
| NULL|
| NULL|
| NULL|
| NULL|
| NULL|
| NULL|
| NULL|
| NULL|
| NULL|
+-----+
only showing top 20 rows

+--------+
|quantity|
+--------+
|    NULL|
|    NULL|
|    NULL|
|    NULL|
|    NULL|
|    NULL|
|    NULL|
|    NULL|
|    NULL|
|    NULL|
|    NULL|
|    NULL|
|    NULL|
|    NULL|
|    NULL|
|    NULL|
|    NULL|
|    NULL|
|    NULL|
|    NULL|
+--------+
only showing top 20 rows



Invalid Date Format Check

In [ ]:
orders_df.filter(col("order_timestamp").isNull()).show()


+--------+-------+---------------+------+
|order_id|user_id|order_timestamp|amount|
+--------+-------+---------------+------+
+--------+-------+---------------+------+



Create Data Quality Summary Report

In [ ]:
dq_summary = {
    "Users_Rows": users_df.count(),
    "Products_Rows": products_df.count(),
    "Orders_Rows": orders_df.count(),
    "Order_Items_Rows": order_items_df.count()
}

dq_summary


{'Users_Rows': 4202,
 'Products_Rows': 629,
 'Orders_Rows': 9269,
 'Order_Items_Rows': 21520}

Save Clean CSV Files Separately

In [ ]:
dim_users.write.mode("overwrite").csv("/content/dim_users_clean")
dim_products.write.mode("overwrite").csv("/content/dim_products_clean")
orders_df.write.mode("overwrite").csv("/content/orders_clean")
order_items_df.write.mode("overwrite").csv("/content/order_items_clean")


Zip and Download Files

In [ ]:
import shutil
from google.colab import files

shutil.make_archive("dim_users_clean", "zip", "/content/dim_users_clean")
shutil.make_archive("dim_products_clean", "zip", "/content/dim_products_clean")
shutil.make_archive("orders_clean", "zip", "/content/orders_clean")
shutil.make_archive("order_items_clean", "zip", "/content/order_items_clean")

files.download("dim_users_clean.zip")
files.download("dim_products_clean.zip")
files.download("orders_clean.zip")
files.download("order_items_clean.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>